In [112]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')


In [113]:
movies=pd.read_csv("D:\\Artificial intelligence\\IA 2023 -\\Machine Learning\\Sklearn\\Recommander System\\10000 Movies Data\\10000 Movies Data")
credits=pd.read_csv('D:\\Artificial intelligence\\IA 2023 -\\Machine Learning\\Sklearn\\Recommander System\\10000 Credits Data\\10000 Credits Data')


movies.head(2)

,Unnamed: 0,Movie_id,title,Genres,release_date,Keywords,overview,poster_path,Budget,Revenue,popularity,vote_average,vote_count
0,0,238,The Godfather,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1972-03-14,"[{'id': 131, 'name': 'italy'}, {'id': 697, 'na...","Spanning the years 1945 to 1955, a chronicle o...",/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,245066411,245066411,93.552,8.7,16814
1,1,278,The Shawshank Redemption,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1994-09-23,"[{'id': 378, 'name': 'prison'}, {'id': 417, 'n...",Framed in the 1940s for the double murder of h...,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,28341469,28341469,78.664,8.7,22542


In [114]:
credits=credits.drop("title",axis=1)

In [115]:

movies=movies.merge(credits,on='Movie_id')


In [116]:
#Final dataset shape :
movies.shape

(9980, 16)

In [117]:
movies.head(2)

,Unnamed: 0_x,Movie_id,title,Genres,release_date,Keywords,overview,poster_path,Budget,Revenue,popularity,vote_average,vote_count,Unnamed: 0_y,Cast,Crew
0,0,238,The Godfather,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1972-03-14,"[{'id': 131, 'name': 'italy'}, {'id': 697, 'na...","Spanning the years 1945 to 1955, a chronicle o...",/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,245066411,245066411,93.552,8.7,16814,0,"[{'adult': False, 'gender': 2, 'id': 3084, 'kn...","[{'adult': False, 'gender': 2, 'id': 154, 'kno..."
1,1,278,The Shawshank Redemption,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",1994-09-23,"[{'id': 378, 'name': 'prison'}, {'id': 417, 'n...",Framed in the 1940s for the double murder of h...,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,28341469,28341469,78.664,8.7,22542,1,"[{'adult': False, 'gender': 2, 'id': 504, 'kno...","[{'adult': False, 'gender': 2, 'id': 153, 'kno..."


In [118]:
# now convert those string list  item into list
import ast
ast.literal_eval

<function ast.literal_eval(node_or_string)>

In [119]:
def convert(obj):
    L =[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [120]:
movies['Keywords']=movies['Keywords'].apply(convert)
movies['Genres']=movies['Genres'].apply(convert)


In [121]:
#movies_cast name
def convert3(obj):
    L =[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3: # takes maximum 3 cast members
            L.append(i['name'])
            counter+=1
    return L

In [122]:
movies['Cast']=movies['Cast'].apply(convert3)


In [123]:
# this will takes only director from the crew lists
def fetch_directoer(obj):
    L =[]
    for i in ast.literal_eval(obj):
        if i['job']=="Director":
            L.append(i['name'])
            break
    return L

In [124]:
movies['Crew']=movies['Crew'].apply(fetch_directoer)

In [125]:
selected_features = ['Keywords','Genres','Cast','Crew']

movies[selected_features].isna().sum().sort_values(ascending=False)


Keywords    0
Genres      0
Cast        0
Crew        0
dtype: int64

In [126]:
movies['tags']= movies['Genres'] + movies['Keywords'] + movies['Cast'] + movies['Crew'] 
movies.head(3)

,Unnamed: 0_x,Movie_id,title,Genres,release_date,Keywords,overview,poster_path,Budget,Revenue,popularity,vote_average,vote_count,Unnamed: 0_y,Cast,Crew,tags
0,0,238,The Godfather,"[Drama, Crime]",1972-03-14,"[italy, loss of loved one, love at first sight...","Spanning the years 1945 to 1955, a chronicle o...",/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,245066411,245066411,93.552,8.7,16814,0,"[Marlon Brando, Al Pacino, James Caan]",[Francis Ford Coppola],"[Drama, Crime, italy, loss of loved one, love ..."
1,1,278,The Shawshank Redemption,"[Drama, Crime]",1994-09-23,"[prison, corruption, police brutality, based o...",Framed in the 1940s for the double murder of h...,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,28341469,28341469,78.664,8.7,22542,1,"[Tim Robbins, Morgan Freeman, Bob Gunton]",[Frank Darabont],"[Drama, Crime, prison, corruption, police brut..."
2,2,240,The Godfather Part II,"[Drama, Crime]",1974-12-20,"[italy, italian american, cuba, symbolism, gan...",In the continuing saga of the Corleone crime f...,/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,102600000,102600000,55.752,8.6,10187,2,"[Al Pacino, Robert Duvall, Diane Keaton]",[Francis Ford Coppola],"[Drama, Crime, italy, italian american, cuba, ..."


In [127]:
cleaned_df = movies[['Movie_id','title','tags']]
cleaned_df['tags']=cleaned_df['tags'].apply(lambda x: " ".join(x))
cleaned_df['tags'] = cleaned_df['tags'].str.lower()

cleaned_df.head(2)
tags=cleaned_df['tags']
tags

0       drama crime italy loss of loved one love at fi...
1       drama crime prison corruption police brutality...
2       drama crime italy italian american cuba symbol...
3       comedy drama romance shah rukh khan kajol amri...
4       drama history war based on novel or book facto...
                              ...                        
9975    action crime thriller crime boss heist bruce w...
9976    horror monster darkness desert archaeologist f...
9977    comedy family tv movie christmas jodelle ferla...
9978    action competition career sports automobile sy...
9979    horror poltergeist haunted house remake during...
Name: tags, Length: 9980, dtype: object

In [144]:
cleaned_df.duplicated().sum()


0

### TEXT Vectorization

In [128]:
vectorizer =TfidfVectorizer()
feature_vectors=vectorizer.fit_transform(tags)
feature_vectors.shape

(9980, 23377)

### Cosine similarity

In [129]:
# get the similarity scores
similarity = cosine_similarity(feature_vectors)


In [130]:
print(similarity)

[[1.         0.06343107 0.35104842 ... 0.0073161  0.         0.        ]
 [0.06343107 1.         0.01815827 ... 0.         0.         0.        ]
 [0.35104842 0.01815827 1.         ... 0.         0.         0.        ]
 ...
 [0.0073161  0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [131]:
similarity.shape


(9980, 9980)

In [132]:

def aggregate_features(selected_movie_indices, feature_vectors):
    # Assuming selected_movie_indices is a list of indices for the selected movies
    selected_features = feature_vectors[selected_movie_indices]
    
    # Choose an aggregation strategy, e.g., taking the mean
    aggregated_features = np.mean(selected_features, axis=0)
    
    return aggregated_features

In [133]:
def collaborative_filtering(selected_movie_indices, feature_vectors, movies):
    # Aggregate features for all movie
    aggregated_features = aggregate_features(selected_movie_indices, feature_vectors)
    
    # Calculate cosine similarity with all movies
    similarity = cosine_similarity(np.asarray(aggregated_features).reshape(1, -1), feature_vectors).flatten()

 
    sorted_indices = np.argsort(similarity)[::-1] #descendant sorting
    recommended_movies = movies['title'][sorted_indices]

    return recommended_movies

In [156]:
selected_movies =[]

## Recommandation based on multiple movies
#### Rerun this cell if you want to add another movie 
##### You can be a little bit approximative in your input

In [159]:
import difflib



movie_name = input('Enter the movie name u like')
find_match= difflib.get_close_matches(movie_name,movies['title'])# For approximative inputs
if len(find_match) >0:
    close_match=find_match[0]
    print(f'close match : {close_match}')

    selected_movies.append(close_match)
    selected_movie_indices = [movies[movies['title'] == movie].index[0] for movie in selected_movies]

    result = collaborative_filtering(selected_movie_indices, feature_vectors, movies)
else:
    print(f'sorry no matching movies for : {movie_name} ')
print(f'selected movies are : {selected_movies}')

print(f'20 recommanded movies : \n{result[:20]}')


close match : Shutter Island
selected movies are : ['Spider-Man', 'Toy Story', 'Shutter Island']
20 recommanded movies : 
2297                      Spider-Man
361                        Toy Story
151                   Shutter Island
2399                    Spider-Man 2
6367                    Spider-Man 3
1189                     Toy Story 4
5614                  Small Soldiers
1090                     Toy Story 2
1771                The Great Gatsby
635                      Toy Story 3
848                    Batman Begins
6233      The Indian in the Cupboard
5343               Welcome to Marwen
7275                I Am Number Four
1287    Batman: Mask of the Phantasm
4852          The Amazing Spider-Man
9939                            Toys
14                   The Dark Knight
1664                  The Lego Movie
1778          The Basketball Diaries
Name: title, dtype: object


## Recommandation based on one movie

In [160]:
def recommandation(movie_name):
    index=(np.where(movies['title']==movie_name)[0])
    if(len(index)==0 ):
        print('sorry no movies matching ')
    else:
        indices_descending = np.argsort(similarity[index])[0][::-1]
        
        # Use the indices to get the sorted array in descending order
        recommanded_movies = movies['title'][indices_descending]
        return recommanded_movies

In [163]:
movie_name = input('Enter the movie name u like')
find_match= difflib.get_close_matches(movie_name,movies['title'])

print(find_match)
if len(find_match) >0:
    close_match=find_match[0]
    print(f'movie found : {close_match}')
    res= recommandation(close_match)
    print(res[:10])
else:
    print('Sorr no matching movie on our dataset')


['Shutter Island', 'Cutthroat Island']
movie found : Shutter Island
151                   Shutter Island
1771                The Great Gatsby
469                        Gone Girl
5138                    Body of Lies
2599                            Hugo
3376                   The Big Shave
7565                       Galveston
2602    Murder on the Orient Express
2395                     The Aviator
1973                 Minority Report
Name: title, dtype: object
